In [28]:
import numpy as np
import numpy.linalg as la
from graphviz import Digraph

In [29]:
m = 5
k = 100
n = 10000
matrix = [[0 for i in range(m)] for j in range(m)]
matrix[0][1] = 0.6
matrix[1][2] = 0.2
matrix[2][3] = 0.3
matrix[3][4] = 0.4
matrix[4][0] = 0.5

In [30]:
for i in range(m):
    matrix[i][i] = 1 - sum(matrix[i])
print(np.array(matrix))

[[0.4 0.6 0.  0.  0. ]
 [0.  0.8 0.2 0.  0. ]
 [0.  0.  0.7 0.3 0. ]
 [0.  0.  0.  0.6 0.4]
 [0.5 0.  0.  0.  0.5]]


In [31]:
def plot_graph(matrix):
    dot = Digraph(format='png', engine='dot')
    num_states = len(matrix)

    for i in range(num_states):
        dot.node(str(i), f"State {i + 1}")

    for i in range(num_states):
        for j in range(num_states):
            prob = matrix[i][j]
            if prob > 0:
                dot.edge(str(i), str(j), label=f"{prob:.2f}")

    dot.render('markov_chain', view=True)

#plot_graph(matrix)

In [32]:
matrix = np.array(matrix)
print(matrix)
np.set_printoptions(precision=3, suppress=True)
stationary_matrix = np.copy(matrix)
for i in range(100):
    stationary_matrix = stationary_matrix @ matrix
print("matrix:", stationary_matrix)
print("matrix2:", stationary_matrix@matrix)

[[0.4 0.6 0.  0.  0. ]
 [0.  0.8 0.2 0.  0. ]
 [0.  0.  0.7 0.3 0. ]
 [0.  0.  0.  0.6 0.4]
 [0.5 0.  0.  0.  0.5]]
matrix: [[0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]]
matrix2: [[0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]]


In [33]:
def generate_initial_state(num_states, verbose=False):
    r = np.random.rand(num_states - 1)
    r = np.sort(r)
    r = np.append(np.array([0]), r)
    r = np.append(r, np.array([1]))
    if verbose:
        print("generated r:", r)
    r = np.diff(r)
    if verbose:
        print("differentiated r:", r)
    return r

initial_state = generate_initial_state(m, verbose=True)

generated r: [0.    0.014 0.315 0.582 0.765 1.   ]
differentiated r: [0.014 0.3   0.268 0.183 0.235]


In [34]:
p_k = initial_state @ la.matrix_power(matrix, k)
print(p_k)

[0.115 0.345 0.23  0.172 0.138]


In [35]:
def roll_state(p: list, verbose: False) -> int:
    if verbose:
        print("probabilities:", p)
    cumulated_p = np.cumsum(p)
    if verbose:
        print("cumulated p:", cumulated_p)
    k = np.random.rand(1)
    if verbose:
        print("random num:", k)
    for i, el in enumerate(cumulated_p):
        if k < el:
            if verbose:
                print("->", i+1, "state")
            return i 
    

In [36]:
trajectories = np.zeros((n, k + 1)) # timestamps {0,...,k}; trajectories {0,...,n - 1}
p_states = np.zeros((n, k + 2, m)) # states {0 (initial),..., k+1 (on k step)}
p_states[:, 0, :] = initial_state
print("initial state:", initial_state)
for i in range(n):
    for j in range(k + 1):
        if i == 0:
            verb = True
        else:
            verb = False
        next_state = roll_state(p_states[i][j], verbose=verb) # index {0...m-1}
        p_states[i, j + 1, :] = matrix[next_state]
        trajectories[i, j] = next_state + 1
print(trajectories[0, :])

initial state: [0.014 0.3   0.268 0.183 0.235]
probabilities: [0.014 0.3   0.268 0.183 0.235]
cumulated p: [0.014 0.315 0.582 0.765 1.   ]
random num: [0.74]
-> 4 state
probabilities: [0.  0.  0.  0.6 0.4]
cumulated p: [0.  0.  0.  0.6 1. ]
random num: [0.154]
-> 4 state
probabilities: [0.  0.  0.  0.6 0.4]
cumulated p: [0.  0.  0.  0.6 1. ]
random num: [0.219]
-> 4 state
probabilities: [0.  0.  0.  0.6 0.4]
cumulated p: [0.  0.  0.  0.6 1. ]
random num: [0.795]
-> 5 state
probabilities: [0.5 0.  0.  0.  0.5]
cumulated p: [0.5 0.5 0.5 0.5 1. ]
random num: [0.372]
-> 1 state
probabilities: [0.4 0.6 0.  0.  0. ]
cumulated p: [0.4 1.  1.  1.  1. ]
random num: [0.557]
-> 2 state
probabilities: [0.  0.8 0.2 0.  0. ]
cumulated p: [0.  0.8 1.  1.  1. ]
random num: [0.17]
-> 2 state
probabilities: [0.  0.8 0.2 0.  0. ]
cumulated p: [0.  0.8 1.  1.  1. ]
random num: [0.401]
-> 2 state
probabilities: [0.  0.8 0.2 0.  0. ]
cumulated p: [0.  0.8 1.  1.  1. ]
random num: [0.194]
-> 2 state
probabil

In [37]:
_, state_counts = np.unique(trajectories[:, k], return_counts=True)
p_states_on_k_step = state_counts / sum(state_counts)
print("empirical:", p_states_on_k_step)
print("theoretical:", p_k)

empirical: [0.122 0.349 0.226 0.17  0.133]
theoretical: [0.115 0.345 0.23  0.172 0.138]


In [38]:
SLAU = np.empty((m, m))
b = np.empty(m)
for i in range(m - 1):
    for j in range(m):
        SLAU[i, j] = matrix[j][i]
    b[i] = 0
    SLAU[i, i] -= 1
# normalizing
for i in range(m):
    SLAU[-1, i] = 1
b[-1] = 1
print("SLAU:", SLAU)
print("b:", b)

SLAU: [[-0.6  0.   0.   0.   0.5]
 [ 0.6 -0.2  0.   0.   0. ]
 [ 0.   0.2 -0.3  0.   0. ]
 [ 0.   0.   0.3 -0.4  0. ]
 [ 1.   1.   1.   1.   1. ]]
b: [0. 0. 0. 0. 1.]


In [39]:
stationary_probs = la.solve(SLAU, b)
print(stationary_probs)
print(p_k)

[0.115 0.345 0.23  0.172 0.138]
[0.115 0.345 0.23  0.172 0.138]


In [40]:
print(stationary_matrix)

[[0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]
 [0.115 0.345 0.23  0.172 0.138]]
